# yo-yo-maskr
## Part 1: Regex based anonymization

In [2]:
from tqdm.auto import tqdm
from src.utils.regex import anonymize_dates, anonymize_entities, create_names_regex

with open('./data/first_names.txt') as f:
    first_names = f.read().split('\n')

with open('./data/last_names.txt') as f:
    last_names = f.read().split('\n')

with open('./sample_texts.txt') as f:
    sample_texts = f.readlines()

result = [anonymize_entities(text, by_names='NAME', first_names=first_names, last_names=last_names) for text in tqdm(sample_texts[:])]
for i in range(len(result[:30])):
    print(result[i])

  0%|          | 0/2194 [00:00<?, ?it/s]

{'text': 'ggsdgg\n', 'replace_dict': {}}
{'text': 'rien à ajouter\n', 'replace_dict': {}}
{'text': 'Alles bestens abgelaufen. Sehr freundlicher Kundenkontakt.\n', 'replace_dict': {}}
{'text': 'Chez la bâloise on Présume coupable et fraudeur\n', 'replace_dict': {}}
{'text': 'Schnelle unkomplizierte Erledigung\n', 'replace_dict': {}}
{'text': 'Ich wurde in Mallorca bestohlen. Habe alles reibungslos mit dem Schadenexperten am #DATE_1# besprochen. Er war wirklich sehr kompetent und freundlich.  Auch die Auszahlung war nach meiner eingereichter Aufstellung super schnell und grosszügig.  Besten Dank.\n', 'replace_dict': {'14.Mai': '#DATE_1#'}}
{'text': 'Unkomplizierte und zeitgemässe Abhandlung des Schadenfalls.\n', 'replace_dict': {}}
{'text': 'très satisfaite de traitement de sinistre, rapide et efficace\n', 'replace_dict': {}}
{'text': 'Kompetent, unkompliziert und immer freundlich!  #NAME_2# #NAME_1# ist immer sehr hilfsbereit und man merkt, dass ihm seine Kunden sehr wichtig sind!!\n', 